In [1]:
# pip3 install opencv-python numpy matplotlib
import cv2 as cv
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt
import os


C:\ProgramData\Anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [7]:
# Source: https://medium.com/analytics-vidhya/binarization-of-image-using-numpy-65df2b82e189
def read_this(image_file, gray_scale=False):
    image_src = cv.imread(image_file)
    if gray_scale:
        image_src = cv.cvtColor(image_src, cv.COLOR_BGR2GRAY)
    else:
        image_src = cv.cvtColor(image_src, cv.COLOR_BGR2RGB)
    return image_src

In [8]:
def ApplyOtsu(image_file, thresh_val=127, gray_scale=True):
    image_src = read_this(image_file=image_file, gray_scale=gray_scale)
    blur = cv.GaussianBlur(image_src,(5,5),0)
    th, image_b = cv.threshold(image_src, 0, 255, cv.THRESH_BINARY+ cv.THRESH_OTSU)
    return Image.fromarray(image_b)

In [9]:
def save_image(image, old_name, dir):
    if('_binarized' not in old_name):
        index = dir.find('.jpg')
        final_string = dir[:index] + '_binarized' + dir[index:]
        image.save(final_string)
    

def main():
    dir = os.getcwd()
    try:
        #assuming this file is one directory upper than images
        directory = os.fsencode(dir + '/PatRec17_KWS_Data/images/')
        for file in os.listdir(directory):
            filename = os.fsdecode(file)
            if filename.endswith(".jpg"):
                image_dir = dir + '/PatRec17_KWS_Data/images/' + filename
                binarized_image = ApplyOtsu(image_file=image_dir)
                save_image(binarized_image, filename, image_dir)
            else:
                continue
                
    except Exception as e:
        print(e)

if __name__ == "__main__":
    main()  

In [ ]:
# Applies KMeans clustering with the given k on the image at the given path 
# and returns the output as a a PIL.Image.
def ApplyKMeansClusteringToImageFile(jpg_image_filename, k=2):
  original_image = cv.imread(jpg_image_filename)
  pixels = original_image.reshape((-1,3))
  pixels = np.float32(pixels)

  # Define criteria, number of clusters and apply KMeans.
  criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 10, 1.0)
  _, label, center = cv.kmeans(pixels, k, None, criteria, 10, cv.KMEANS_RANDOM_CENTERS)

  # Convert back into uint8.
  center = np.uint8(center)
  final_image_pixels = center[label.flatten()]
  final_image_pixels = final_image_pixels.reshape((original_image.shape))

  return Image.fromarray(final_image_pixels)




In [ ]:
# IN PROGRESS, NOT WORKING/FINISHED. Probably not worth finishing anyway.
# - original_image: PIL.Image
# https://www.thepythoncode.com/article/kmeans-for-image-segmentation-opencv-python
# https://medium.com/towardssingularity/k-means-clustering-for-image-segmentation-using-opencv-in-python-17178ce3d6f3
def ApplyKMeansClusteringToImagePixels(original_image, k=2):
  pixels = []
  for row in range(original_image.height):
    pixel_row = []
    for column in range(original_image.width):
      pixel_row.append(original_image.getpixel((column, row)))
    pixels.append(pixel_row)
  pixels = np.float32(pixels)
  pixel_vals = pixels.reshape((-1,3)) 

  x_size, y_size = original_image.size
  shape = (x_size, y_size, 1)

  # processed_image = Image.new('L', original_image.size, 0)
  # return processed_image

  criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 100, 0.85) #criteria
  _, labels, centers = cv.kmeans(pixels, k, None, criteria, 10, cv.KMEANS_RANDOM_CENTERS) 

  centers = np.uint8(centers) # convert data into 8-bit values 
  segmented_data = centers[labels.flatten()] # Mapping labels to center points( RGB Value)
  segmented_image = segmented_data
  # segmented_image = segmented_data.reshape((shape)) # reshape data into the original image dimensions
  plt.imshow(segmented_image)
  plt.show()
  input()